In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np
import tflearn
import tensorflow as tf
import random

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


In [2]:
# import our chat-bot intents file
import json
with open('../data/intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
words = []
classes = []
documents = []
ignore_words = ['?']
response = {}
for intent in intents['intents']:
    for pattern in intent['patterns']:
        if pattern is None:
            pattern = "nothing specified"
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
    response[intent["tag"]] = intent["responses"]

words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

398 documents
7 classes ['emotional', 'greeting', 'history', 'incident', 'politics', 'profile', 'thanks']
605 unique stemmed words ['!', '#', "'m", "'s", '(', ')', ',', '-', '.', '...', '//infonet.na.frs/departments/gss/gsshome/default.aspx', '//portal.lumension.com', '//upload-na.ckggsoftware.com', '100mbit', '10400', '14805', '2015', '2016', '2017', '23', '2nd', '31', '400', '42', '51231', '565355', ':', ';', 'a', 'about', 'acceiv', 'access', 'account', 'act', 'ad', 'adapt', 'admin', 'advoc', 'afraid', 'aft', 'ag', 'again', 'al', 'alert', 'alex', 'am', 'amend', 'amendemnt', 'amus', 'an', 'and', 'andrew', 'angry', 'answ', 'any', 'anyon', 'anyth', 'appl', 'apply', 'approv', 'ar', 'arrog', 'as', 'asham', 'ask', 'at', 'attach', 'august', 'autom', 'av', 'away', 'az', 'b.itdog.com', 'bab', 'back', 'badg', 'batist', 'be', 'been', 'behalf', 'between', 'bhanu', 'bil', 'bin', 'blam', 'blu', 'bor', 'boss', 'bot', 'box', 'bracknel', 'broad', 'brok', 'broth', 'build', 'built', 'but', 'buy', 'by',

In [4]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [15]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=60, batch_size=8, show_metric=True)
model.save('../data/model.tflearn')

Training Step: 2999  | total loss: 0.09429 | time: 0.091s
| Adam | epoch: 060 | loss: 0.09429 - acc: 0.9990 -- iter: 392/398
Training Step: 3000  | total loss: 0.09072 | time: 0.092s
| Adam | epoch: 060 | loss: 0.09072 - acc: 0.9991 -- iter: 398/398
--
INFO:tensorflow:/Users/tuhin.sharma/Documents/Git/chatbot-incident-ticket/chatbot_platform/data/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [7]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [8]:
print(model.predict([p]))

[[2.9185266e-04 3.0073694e-05 6.0774753e-04 6.3243622e-01 1.0114135e-03
  2.9661033e-01 6.9012448e-02]]


In [16]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "../data/training_data", "wb" ) )

In [13]:
len(train_x[0])

605

In [14]:
len(p)

605